## Amazon movie reviews data cleanup
Using spark for pre-processing

In [1]:
import pyspark as ps    # for the pyspark suite
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('moviemood')
         .getOrCreate()
        )

In [2]:
data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [3]:
amazon_reviews_df = spark.read.json(data_path + file_name)

In [4]:
amazon_reviews_df.cache()
amazon_reviews_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [5]:
total_comment_count = amazon_reviews_df.count()

In [6]:
total_comment_count

4607047

## Data sampling for development

In [200]:
dev_sample_fraction = 0.00001

In [201]:
reviews_spark_df = amazon_reviews_df.sample(withReplacement=False, fraction=dev_sample_fraction)

In [202]:
reviews_df = reviews_spark_df.toPandas()

/anaconda3/lib/python3.7/socket.py:660: ResourceWarning: unclosed <socket.socket fd=76, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 61494), raddr=('127.0.0.1', 61493)>
  self._sock = None


In [203]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 9 columns):
asin              49 non-null object
helpful           49 non-null object
overall           49 non-null float64
reviewText        49 non-null object
reviewTime        49 non-null object
reviewerID        49 non-null object
reviewerName      48 non-null object
summary           49 non-null object
unixReviewTime    49 non-null int64
dtypes: float64(1), int64(1), object(7)
memory usage: 3.5+ KB


In [204]:
reviews_df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0780628551,"[0, 1]",4.0,"Most viewers said this film was bad, but to me it was pretty good. I don't usually get sci-fic movies but I actually get this one. If you like horror, love, and space movies check this movie out. This movie is about a perfect all-American couple. Spencer Armocost (Depp)has died in space, then a life force take his body and his memory. NASA lost contact for two minutes and in space with the another astronaut contact them back. His goal to make his wife(Theron) pregant and he need his own kind to make a space ship from their design. ~~~~~I ACTUALLY GIVE IT 4.5 STARS!~~~~~","03 14, 2000",A2FDCJXVKQ39OM,Amy Liu,The best Sci-fic movie I've ever seen!,952992000
1,0792833171,"[0, 0]",5.0,What can I say that possibly hasn't been said about this movie over the years? It is brilliant and should be required watching. The digital download was flawless and worked perfectly.,"01 2, 2013",A1MCY9RWSGNNZ9,Anthony Sacco,A classic.,1357084800
2,1573622990,"[0, 0]",5.0,"this is a must, the last statement at the end, a few sentences changed, changes and reminds me of my life andall things large and small and happy and sad to be thankful for...a must see!","08 2, 2013",A1NMPROID1MEEV,Patricia A. Hayes,important love story,1375401600
3,6301661729,"[1, 1]",5.0,"Perhaps the single greatest tribute to this film is that Tim Burton paid homage to it with a remake that was good but failed to capture the impact of the original. It is hard to identify exactly why this film seized the imagination of several generations especially since in many ways &quot;Planet of the Apes&quot; was a commentary on the sixties. I saw it and learned, as I drove away from the theater, that Rev. Martin Luther King, Jr. had just been shot. The poignant coincidence of events will never be forgotten. This is one of the great science fiction films with a remarkable script by the dean of fantasy writers Rod Serling. It is well acted, even behind the make-up. It has one of Jerry Goldsmith's best and most original scores. And it still holds up quite well, largely because it was not driven by special effects. The new edition does not offer much beyond the widescreen version of the DVD, but the extras are terrific and make this a worthy purchase. Charlton Heston gave one of his best perfomances and will always be remembered for two scenes...when he regains his ability to speak (you know the line), and of course the end scene that still works and may well prove to be a testament to humankind in the modern age.","02 17, 2004",A2VOIE123XM7T5,G M. Stathis,This Edition Goes Ape,1076976000
4,6301773551,"[15, 24]",1.0,"I could write so much on this subject... I'll just stick with the review of this new DVD.YOU DON'T RELEASE A MOVIE IN 4:3 LETTERBOX IN THIS DAY AND AGE! George Lucas was the one pushing for all-digital theatres and all that. And now he disrespects the work of all the people that helped him make Star Wars a success in the first place back in 1977.I bought a DVD player originally so that I would own the original Star Wars on DVD. I got an HDTV compatible widescreen TV to see it in at least standard 16x9 anamorphic resolution. (like 94 % of big movie DVD`s out there..)Now I realise.. After all those years of waiting, George Lucas is sooooo thick... He just WON'T give fans what THEY really want... All those people who made him rich.Go to hell, George Lucas. You have lost my admiration for good.","09 25, 2006",A2QGFZCM6RIX7P,"Felix A. Mackay ""Felix""",How George Lucas killed Star Wars,1159142400


### Filter out comments that may be relative to the support (DVD, edition...)

In [205]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

words_to_remove = ['dvd', 'vhs','edition', 'blue-ray', 'blueray', 'blu-ray', 'bluray', 'price']

In [206]:
words_to_remove_re = '(' + '|'.join(words_to_remove) + ')'

In [207]:
import warnings
warnings.simplefilter('ignore')
text_removed = reviews_df[reviews_df['reviewText'].str.contains(words_to_remove_re, case=False)]
warnings.simplefilter('default')

In [208]:
text_removed.describe()

,overall,unixReviewTime
count,13.000000,1.300000e+01
mean,3.923077,1.213674e+09
std,1.605280,1.006942e+08
min,1.000000,1.076976e+09
25%,3.000000,1.157933e+09
50%,5.000000,1.184976e+09
75%,5.000000,1.257811e+09
max,5.000000,1.388448e+09


#### Result: fraction of content removed:

In [209]:
text_removed['reviewText'].count() / (total_comment_count * dev_sample_fraction)

0.2821764136549942

## Quality check on filtered data

In [210]:
warnings.simplefilter('ignore')
relevant_comments = reviews_df[~reviews_df['reviewText'].str.contains(words_to_remove_re, case=False)]
warnings.simplefilter('default')

In [211]:
relevant_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 48
Data columns (total 9 columns):
asin              36 non-null object
helpful           36 non-null object
overall           36 non-null float64
reviewText        36 non-null object
reviewTime        36 non-null object
reviewerID        36 non-null object
reviewerName      35 non-null object
summary           36 non-null object
unixReviewTime    36 non-null int64
dtypes: float64(1), int64(1), object(7)
memory usage: 2.8+ KB


In [212]:
relevant_comments['reviewText'].head(2) # try head(50) or more

0    Most viewers said this film was bad, but to me it was pretty good. I don't usually get sci-fic movies but I actually get this one. If you like horror, love, and space movies check this movie out. This movie is about a perfect  all-American couple. Spencer Armocost (Depp)has died in space, then a life  force take his body and his memory. NASA lost contact for two minutes and  in space with the another astronaut contact them back. His goal to make his  wife(Theron) pregant and he need his own kind to make a space ship from  their design.  ~~~~~I ACTUALLY GIVE IT 4.5 STARS!~~~~~
1    What can I say that possibly hasn't been said about this movie over the years? It is brilliant and should be required watching. The digital download was flawless and worked perfectly.                                                                                                                                                                                                                                

#### Result: reviews look good
Based on quick review of head(50)

## Keyword-based sentiment analysis

In [229]:
emotion_keywords_path = '../../datasets/'
emotion_keywords = 'emotions-sensor-data-set.zip'
global emotions_keywords_df
emotions_keywords_df = pd.read_csv(emotion_keywords_path + emotion_keywords)

In [267]:
# emotions_keywords_df

#### Dictionary for speed

In [272]:
import csv
global emotions
emotions = list(emotions_keywords_df)[1:]
emotion_keywords_csv = 'Andbrain_DataSet.csv'

emotions_keywords = {}
emotions2 = []
with open(emotion_keywords_path + emotion_keywords_csv, mode='r', errors='replace') as csvfile:
    reader = csv.reader(csvfile, dialect='excel')
    emotions2 = reader.__next__()  # skip headers
#     print (emotions2)
    for row in reader:
#         print(row)
        dico = {}
        for i in range(1,8):
            dico[emotions2[i].strip()] = float(row[i])
        emotions_keywords[row[0].strip()] = dico
        
emotions2

['word', 'disgust', 'surprise', 'neutral', 'anger', 'sad', 'happy', 'fear']

In [274]:
emotions_keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 8 columns):
word        1104 non-null object
disgust     1104 non-null float64
surprise    1104 non-null float64
neutral     1104 non-null float64
anger       1104 non-null float64
sad         1104 non-null float64
happy       1104 non-null float64
fear        1104 non-null float64
dtypes: float64(7), object(1)
memory usage: 69.1+ KB


In [217]:
global emotions
emotions = list(emotions_keywords_df)[1:]
rel_comments_w_sentiments = relevant_comments.reindex(columns= relevant_comments.columns.tolist() + emotions)

In [218]:
rel_comments_w_sentiments.head(1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,disgust,surprise,neutral,anger,sad,happy,fear
0,0780628551,"[0, 1]",4.0,"Most viewers said this film was bad, but to me it was pretty good. I don't usually get sci-fic movies but I actually get this one. If you like horror, love, and space movies check this movie out. This movie is about a perfect all-American couple. Spencer Armocost (Depp)has died in space, then a life force take his body and his memory. NASA lost contact for two minutes and in space with the another astronaut contact them back. His goal to make his wife(Theron) pregant and he need his own kind to make a space ship from their design. ~~~~~I ACTUALLY GIVE IT 4.5 STARS!~~~~~","03 14, 2000",A2FDCJXVKQ39OM,Amy Liu,The best Sci-fic movie I've ever seen!,952992000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
import re
comment = rel_comments_w_sentiments['reviewText'][8]
comment

"Perhaps the most underappreciated film of the  80's, the Beast stands out as a solid war epic, with heart, action and beautiful imagery. Set during the Soviet Union's brutal war against Afghanistan, the Beast chronicles one tank's attempts to get back to friendly lines. The conscience of the tank crew is a young soldier played by Jason Patric.  Increasingly put off by the atrocities against civilians that his hardened tank commander (wonderfully played by a rather lean George Dzundza!) has them all commit, he is forced out of the tank and left to die in the Afghan desert.  While waiting to die, he is set upon by the rebels, who spare his life, and accept his aid in going after his comrades' tank! There was a lot to like about the Beast.  Good acting all around.  Fine attention to military detail.  Suspenseful, well-done action scenes. Haunting score by Mark Isham.  And amazing, razor-sharp cinematography by Douglas Milsome. The images of the desert are haunting and  beautifully compos

In [220]:
# TODO: remove stomp words
test_input = 'ability. Ability,abilities'
scores = {key: 0.0 for key in emotions}
print(scores)

{'disgust': 0.0, 'surprise': 0.0, 'neutral': 0.0, 'anger': 0.0, 'sad': 0.0, 'happy': 0.0, 'fear': 0.0}


In [275]:
emotions_keywords['ability']
emotions2

['word', 'disgust', 'surprise', 'neutral', 'anger', 'sad', 'happy', 'fear']

#### Add vector to a comment

In [291]:


def fill_emotions(comment):
    comment_text = comment['reviewText']
    comment_words = comment_text.split(' ')    
    emotion_keywords_hits = 0
    scores = {key: 0.0 for key in emotions}    # {'disgust': 0.0, ...}
    
    for word in comment_words:
        if emotions_keywords.get(word, 0.0) != 0.0:
            for key in scores.keys():
                scores[key] += emotions_keywords[word][key]
            
    for emotion in emotions:
        comment[emotion] = scores[emotion]
    return comment

rel_comments_w_emotions = rel_comments_w_sentiments.apply(fill_emotions, axis=1, raw=True)

In [292]:
# function test - OK
test_comment = {key: float('nan') for key in emotions}
test_comment['reviewText'] = 'ability able abuse aaa bbb'
fill_emotions(test_comment)

{'disgust': 0.0050135859999999996,
 'surprise': 0.048190635999999995,
 'neutral': 0.001224,
 'anger': 0.161135579,
 'sad': 0.014852553000000001,
 'happy': 0.017783056,
 'fear': 0.043023212,
 'reviewText': 'ability able abuse aaa bbb'}

In [293]:
rel_comments_w_emotions.head(2)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,disgust,surprise,neutral,anger,sad,happy,fear
0,0780628551,"[0, 1]",4.0,"Most viewers said this film was bad, but to me it was pretty good. I don't usually get sci-fic movies but I actually get this one. If you like horror, love, and space movies check this movie out. This movie is about a perfect all-American couple. Spencer Armocost (Depp)has died in space, then a life force take his body and his memory. NASA lost contact for two minutes and in space with the another astronaut contact them back. His goal to make his wife(Theron) pregant and he need his own kind to make a space ship from their design. ~~~~~I ACTUALLY GIVE IT 4.5 STARS!~~~~~","03 14, 2000",A2FDCJXVKQ39OM,Amy Liu,The best Sci-fic movie I've ever seen!,952992000,0.006679,0.131175,0.002613,0.115783,0.12956,0.216092,0.265653
1,0792833171,"[0, 0]",5.0,What can I say that possibly hasn't been said about this movie over the years? It is brilliant and should be required watching. The digital download was flawless and worked perfectly.,"01 2, 2013",A1MCY9RWSGNNZ9,Anthony Sacco,A classic.,1357084800,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000


### Previous attempt too slow with dataframes:

In [225]:
def fill_emotions(comment):

    comment_text = comment['reviewText']    
    emotion_keywords_hits = 0
    scores = {key: 0.0 for key in emotions}    # {'disgust': 0.0, ...}
    for row in emotions_keywords_df.itertuples():
#         print(row)
        word = row.word.strip()  # there are spaces at end
        word_re = r'\b' + re.escape(word) + r'\b'
        match = re.findall(word_re, comment_text,re.IGNORECASE)
        count = len(match)
        if count > 0:
    #         print(match)
            scores['disgust'] += row.disgust
            scores['surprise'] += row.surprise
            scores['neutral'] += row.neutral
            scores['anger'] += row.anger
            scores['sad'] += row.sad
            scores['happy'] += row.happy
            scores['fear'] += row.fear
            emotion_keywords_hits += 1

    result = comment.copy()
#     for emotion in emotions:
#         comment[emotion] = scores[emotion]

    # print(rel_comments_w_sentiments.loc[8])
    # print(scores)
    # print(emotion_keywords_hits) 

rel_comments_w_emotions = rel_comments_w_sentiments.apply(fill_emotions, axis=1, raw=True)

In [222]:
rel_comments_w_emotions.head(3)

0    None
1    None
2    None
dtype: object

In [93]:
from compute_emotions import *
#computeEmotions(4)

In [166]:
global g0
g0 = 1
def use_global(string):
    print(g0)
use_global('a')

1


In [167]:
def use_global2(string):
    print(g0)
use_global2('a')

1


## Isolating Comment helpfulness

In [24]:
helpful = reviews_df['helpful'].map(lambda x: x[0], na_action='ignore')
helpful.describe()

count    91869.000000
mean     2.638866    
std      11.521295   
min      0.000000    
25%      0.000000    
50%      0.000000    
75%      2.000000    
max      1031.000000 
Name: helpful, dtype: float64

In [25]:
not_helpful = reviews_df['helpful'].map(lambda x: x[1], na_action='ignore')
not_helpful.describe()

count    91869.000000
mean     4.093655    
std      14.126499   
min      0.000000    
25%      0.000000    
50%      1.000000    
75%      4.000000    
max      1160.000000 
Name: helpful, dtype: float64

### Note: current filtering is probably too aggressive:'DVD' mostly used to describe the movie
Removing DVD would remove a lot of good comments, as all of the following are appropriate:

In [26]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
text_removed['reviewText'].head(5)

11    I have watched this version of &#34;Joseph&#34; numerous times also on TV and am once again glad to add this one to my collection of Bible DVD's. It's entertainment but for me it's much than that I'm able to receive a better understanding of my Bible it's like a learning process to test me on my Bible knowledge and see how well I know the scriptures, for instance when I'm able to pick something out of the movie that may not be quite true to scripture I go check my Bible and I'm able to learn my scriptures better and my Bible. I believe by watching the DVD over and over and even having different versions you absorb more into your memory and it can be your own individual Bible study. I was really amazed and excited when I saw that you offered many of these DVD's of (The Bible Collection) which are some of my favorites. Hoping to collect them all and other versions plus other Christian DVD's to learn from them for a better understanding of God's true Word.
13    I use the Hemisphere

In [28]:
text_removed = reviews_df[(reviews_df['reviewText'].str.contains('(dvd|vhs)', case=False)) & (helpful > 1)]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [29]:
text_removed[['reviewText','helpful']].head(5)

,reviewText,helpful
30,"What with Meryl Streep portraying her in a movie and all, Julia's books and series are getting a monster tidal wave of fresh attention. I wonder how many youngsters (Food Network viewers) will purchase or have purchased for them the classic ""Mastering the Art of French Cooking"" and find it completely overwhelming and not necessarily useful for everyday cooking?I had few cooking skills in the mid 1980s when this book and DVD came out, and I relied on recipes from Fannie Farmer, supermarket recipe cards, and women's magazines. This series elevated my cooking for good. It is French influenced, but simplified in some cases, and there are variations on most recipes which is really useful if you don't have time to run to the supermarket. I have had many variation cookbooks and this is the best. I have not used the dessert section too much (I prefer Pierre Franey for this--fruit salad with Grand Marnier!), but the bread, meat, and veg sections are the best. I recommend ham steaks with peas and mashed potatoes and cornish game hen broiled w/cheese.I am so pleased this is out on DVD. I gathered up the video tapes one-by-one on ebay and this is definitely more affordable!","[31, 34]"
37,"To start off with, I actually have an early release of the dvd, so my stars are based on that, not the cinema.I made it through about six or seven pages on reviews before I decided to write this and from what I see, three subjects usually appear as the reason for bad reviews.The first reason is it is too violent. We all knew this, before we even went to see it. It was all over news, media, etc. We were warned not to take our kids, be prepared for violence, not for the weak of heart. So, if you went expecting scenes of the beach, then that is your own fault. I didn't watch it hoping to see a blood bath, but I knew it was coming.I am a very faithful Protestant, and I do see some Catholic overtones in the story, but that is Mel's interpretation and overall I felt it was pretty accurate. It is interesting to me that all of these people that may or may not have a personal relationship with Jesus, seem to know that if he were here today, he would not support this film.The second reason people complain is because it doesn't tell the full story of Jesus. It isn't supposed to. It is the Passion of the Christ, and passion means suffering. It isn't the story of the Christ or the Gospel of the Christ. It is basically a Passion play put to cinema. He isn't trying to show us the teachings of Jesus, he is showing us the suffering of Jesus. He is showing us what He went through for us. If you see this movie and aren't moved by the sacrifice that He made for all of us to be free, then you don't get it. And if you don't understand and you don't get it, then that is ok, I pray someday that you will.The third thing I pick up on in these reviews is that Mel is using it just for profit. He has made money at ever other movie he has made I am sure, and yet no one ever questions his motives then. Yet, when he comes out with something of a religious nature when we are in a world that requires us to be politically correct, then he was just doing it for money. He put up his own money and I personally hope he makes all the money he can off of it, not that he needs it. So what if there is a director's cut later on? How may different directors editions or box sets of Lord of the Rings are out there? I get tired of having to apologize for what I believe. If someone stands up for an important issue, whatever the hot topic of the week is, then that is their right to voice their opinion and they expect everyone to recieve it and agree. But if I as a Christian stand up for what I believe, then I am a ""Jesus Freak"" or a ""Fanatic"". So if you don't agree with me, that is your right, but if I don't believe what you do, then I am narrow minded?? What happened to my right to my own beliefs?While I am still on my soap box, it is amazing to me that people still think